##### Pipeline

In [1]:
!git clone https://github.com/Muennighoff/cn_performer
%cd cn_performer
!pip3 install -q -r requirements.txt

Cloning into 'cn_performer'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 80 (delta 24), reused 70 (delta 17), pack-reused 0
Unpacking objects: 100% (80/80), done.
/content/cn_performer
     |████████████████████████████████| 1.8MB 13.4MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
     |████████████████████████████████| 81kB 11.5MB/s 
     |████████████████████████████████| 3.2MB 52.9MB/s 
     |████████████████████████████████| 890kB 52.1MB/s 
     |████████████████████████████████| 184kB 47.9MB/s 
     |████████████████████████████████| 2.2MB 54.7MB/s 


##### Preprocessing (Ideally done in the Cloud)

In [ ]:
from google.colab import auth
auth.authenticate_user()
!gcloud init

In [3]:
!mkdir ./data/
!gsutil cp gs://clue-corpus2020/CLUECorpus2020/train_clue_pretrain_train_0000.zip ./data/

mkdir: cannot create directory ‘./data/’: File exists
Copying gs://clue-corpus2020/CLUECorpus2020/train_clue_pretrain_train_0000.zip...

Operation completed over 1 objects/1.0 GiB.                                      


In [4]:
!unzip ./data/*.zip -d ./data/

Archive:  ./data/train_clue_pretrain_train_0000.zip
  inflating: ./data/clue_pretrain_0000000.txt  
  inflating: ./data/clue_pretrain_0000001.txt  
  inflating: ./data/clue_pretrain_0000002.txt  
  inflating: ./data/clue_pretrain_0000003.txt  
  inflating: ./data/clue_pretrain_0000004.txt  
  inflating: ./data/clue_pretrain_0000005.txt  
  inflating: ./data/clue_pretrain_0000006.txt  
  inflating: ./data/clue_pretrain_0000007.txt  
  inflating: ./data/clue_pretrain_0000008.txt  
  inflating: ./data/clue_pretrain_0000009.txt  
  inflating: ./data/clue_pretrain_0000010.txt  
  inflating: ./data/clue_pretrain_0000011.txt  
  inflating: ./data/clue_pretrain_0000012.txt  
  inflating: ./data/clue_pretrain_0000013.txt  
  inflating: ./data/clue_pretrain_0000014.txt  
  inflating: ./data/clue_pretrain_0000015.txt  
  inflating: ./data/clue_pretrain_0000016.txt  
  inflating: ./data/clue_pretrain_0000017.txt  
  inflating: ./data/clue_pretrain_0000018.txt  
  inflating: ./data/clue_pretrain_00

In [5]:
# Metadata
dataset_path = "data"
dataset_name = "clue"
out_name = dataset_name + "_tokenized"

In [6]:
# Launch tokenization & turning into .tfrecords
!python data/create_tfrecords_causal.py --seq_len 512 --processes 2 --files_per 1 --input_dir /content/cn_performer/$dataset_path --output_dir $out_name

2021-02-24 06:11:45.040642: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Downloading: 100% 35.1k/35.1k [00:00<00:00, 415kB/s]
Writing TFRecord Files to clue_tokenized. Parsed 11 input files. files_written : : 36126it [01:33, 396.12it/s]Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 733, in next
    item = self._items.popleft()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "data/create_tfrecords_causal.py", line 195, in create_tfrecords_mp
    for results in pbar:
  File "/usr/local/lib/python3.7/dist-packages/tqdm/std.py", line 1104, in __iter__
    for obj in iterable:
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 737, in next
    self._cond.wait(timeout)
  File "/usr/lib/python3.7/threading.py", line 296, in wait
    waiter.acquire()
KeyboardInterrupt

Du

##### Loading data back in from the Cloud & Training

In [7]:
# Load back in

import tensorflow as tf
import os

def read_tfrecord(name, batch_size=8):
    train_ds = tf.data.TFRecordDataset(name)

    # Describe fts for successful decoding
    # Note we need to add 1 here as we increased the chunk size when writing to account for labels
    txt_feature_description = {
        #'input_ids': tf.io.FixedLenFeature([2049], tf.int64, default_value=[0]*2049),
        #'input_ids': tf.io.FixedLenFeature([1025], tf.int64, default_value=[0]*1025),
        'input_ids': tf.io.FixedLenFeature([513], tf.int64, default_value=[0]*513),
    }

    def _parse_txt_function(example_proto):
      return tf.io.parse_single_example(example_proto, txt_feature_description)

    parsed_ds = train_ds.map(_parse_txt_function)

    parsed_ds = parsed_ds.shuffle(1000).batch(batch_size)

    return parsed_ds

In [8]:
# Load 200MB Files from Cloud Bucket
#FILENAME_PATTERN = f"gs://exp_niklas/clue_tokenized200/*.tfrecords"
FILENAME_PATTERN = "clue_tokenized/*"
filenames = tf.io.gfile.glob(FILENAME_PATTERN)
train_ds = read_tfrecord(filenames, batch_size=1)

# For TPU we need to read them in from the Cloud

In [9]:
import os
import sys
import tensorflow as tf

from transformers import TFGPT2LMHeadModel, GPT2Config

In [10]:
# Train

def create_model(pretrained=False):
    """
    Creates Model
    """

    if pretrained:
      # Seq Len of train_ds must be 1024
      base_model = TFGPT2LMHeadModel.from_pretrained("mymusise/gpt2-medium-chinese", return_dict=False)

    else:
      config = GPT2Config.from_pretrained("mymusise/gpt2-medium-chinese")
      config.n_ctx = 512
      config.n_embd = 512
      config.n_positions = 512

      base_model = TFGPT2LMHeadModel(config)


    model = Wrapper(base_model)
    return model

### Modelling - Static ###

class Wrapper(tf.keras.Model):
    def __init__(self, model, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = model
        # Track loss (Loss itself its CategoricalCrossEnt.)
        self.loss_tracker = tf.keras.metrics.Mean(name='loss') 

    @tf.function
    def train_step(self, data):
        # Data is a dictionary
        x = data["input_ids"][:, :-1]
        y = data["input_ids"][:, 1:]

        print(x)
        print(x.shape)

        with tf.GradientTape() as tape:
            # Forward pass
            y_pred = self.model(x, training=True)
            # Compute the loss value.
            y_pred = y_pred.logits
            loss = self.compute_loss(y, y_pred)
            # Reduce loss to single digit
            loss = tf.reduce_mean(loss)

        # Compute gradients
        gradients = tape.gradient(loss, self.trainable_variables)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        # Update loss tracker
        self.loss_tracker.update_state(loss)  
        # Update metrics
        self.compiled_metrics.update_state(y, y_pred)

        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        # Data is a dictionary
        x = data["input_ids"][:, :-1]
        y = data["input_ids"][:, 1:]

        # Compute predictions
        y_pred = self.model(x, training=False)
        y_pred = y_pred.logits
        loss = self.compute_loss(y, y_pred)

        # Updates the metrics tracking the loss
        self.loss_tracker.update_state(loss)  
        self.compiled_metrics.update_state(y, y_pred)

        return {m.name: m.result() for m in self.metrics}

    def compute_loss(self, labels, logits):
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=tf.keras.losses.Reduction.NONE
        )
        # make sure only labels that are not equal to -100 do affect loss
        active_loss = tf.not_equal(tf.reshape(labels, (-1,)), -100)
        reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
        labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)

        return loss_fn(labels, reduced_logits)
        
def shape_list(tensor: tf.Tensor):
    """
    Deal with dynamic shape in tensorflow cleanly.
    Args:
        tensor (:obj:`tf.Tensor`): The tensor we want the shape of.
    Returns:
        :obj:`List[int]`: The shape of the tensor as a list.

    ### TODO: Move to compute_loss
    """
    dynamic = tf.shape(tensor)

    if tensor.shape == tf.TensorShape(None):
        return dynamic

    static = tensor.shape.as_list()

    return [dynamic[i] if s is None else s for i, s in enumerate(static)]

In [11]:
model = create_model()

learning_rate = 1e-5
optimizer = tf.keras.optimizers.Adam(learning_rate)
model.compile(optimizer=optimizer,
              metrics=['sparse_categorical_accuracy'])

model.fit(train_ds,
          epochs=5)


Epoch 1/5
Tensor("strided_slice:0", shape=(None, 512), dtype=int64)
(None, 512)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet s

KeyboardInterrupt: ignored

In [15]:
from transformers import TextGenerationPipeline, BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("mymusise/gpt2-medium-chinese")
model = create_model(pretrained=True)

text_generater = TextGenerationPipeline(model.model, tokenizer)

texts = [
    '少年对面站着一位中年人，这位中年人两鬓略有些斑白，穿着一套青衫。尽管衣衫有些脱色，但却洗得很干净。',
]

for text in texts:
    print(text_generater(text, max_length=120 + len(text), do_sample=True, top_k=10, no_repeat_ngram_size=3))

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at mymusise/gpt2-medium-chinese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


[{'generated_text': '少年对面站着一位中年人，这位中年人两鬓略有些斑白，穿着一套青衫。尽管衣衫有些脱色，但却洗得很干净。 他 的 双 手 仍 旧 紧 握 在 胸 前 ， 一 个 劲 儿 地 往 后 推 ， “ 哎 呀 ！ ” 我 说 道 ： “ 你 好 啊 ！ 这 真 是 太 可 笑 了 — — 你 要 把 这 件 事 讲 给 别 人 听 ？ 不 过 ， 我 没 办 法 讲 出 来 — — 只 是 ， 我 不 知 道 为 什 么 我 才 想 到 那 些 。 你 要 是 能 够 帮 我 找 到 这 样 的 机 会 呢 — — ” “ 哦 — — 嗯 ， 当 然 啦 ， 当 时 我 不 愿 意 谈'}]
